In [2]:
# boas práticas: criar uma célula com todas as bibliotecas necessárias
# importando as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
# Turmas 2017.2
# http://dados.ufrn.br/dataset/turmas

# turmas
turmas_url = 'http://bit.do/turmas_ufrn_2017_2'

# matrículas em componentes em 2017.2
notas_url = 'http://bit.do/notas_ufrn_2017_2'

# componentes curriculares presenciais
disciplinas_url = 'http://bit.do/componentes_curriculares_ufrn'

# docentes da ufrn atualizado em 2018
docentes_url = 'http://bit.do/docentes_ufrn_2018'

# turmas 2017.2
turmas = pd.read_csv(turmas_url, sep=';')

# matriculas em componentes em 2017.2
notas = pd.read_csv(notas_url, sep=';')

# componentes curriculares presenciais
# por alguma razão ou outra, esse dataset tinha algum lixo em linhas que ocasionava 
# erros de leitura, dessa forma foi realizado um filtro apenas por colunas mais importantes
disciplinas = pd.read_csv(disciplinas_url, 
                          sep=';', 
                          usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])

docentes = pd.read_csv(docentes_url, sep=';')

In [5]:
# Adaptando o dataframe turma e disciplinas para o merge
# Esse procedimento é necessário pois o dataframe turma está codificado com ids

# A coluna id_componente precisa ser a mesma em ambos os dataframes
turmas.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)

# id_componente em ambos os dataframes precisam ser do mesmo tipo
turmas.id_componente = turmas.id_componente.astype(str)

# Limpando o dataframe disciplinas com linhas desnecessárias
disciplinas.dropna(inplace=True)
disciplinas = disciplinas[~disciplinas.id_componente.duplicated()]

In [6]:
# Fazendo um merge entre o dataframe turmas e disciplinas utilizando como base a coluna id_componente
# https://pandas.pydata.org/pandas-docs/stable/merging.html
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

# merge os dois dataframes utilizando como base a coluna id_componenteb
merge_turmas_disciplinas = pd.merge(turmas, disciplinas, on='id_componente')[colunas]

merge_turmas_disciplinas.dropna(inplace=True)
merge_turmas_disciplinas.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,57602957,5753061.0,ART2077,LATO SENSU,PRODUÇÃO COREOGRÁFICA I,DEPARTAMENTO DE ARTES,CONSOLIDADA,40.0,7M2345 7T23456
2,57602962,5752714.0,ART2083,LATO SENSU,VIESES ARTÍSTICOS CONTEMPORÂNEOS EM DANÇA ...,DEPARTAMENTO DE ARTES,ABERTA,40.0,1M345 7M2345 17T23456
3,57602963,3031948.0,ART2086,LATO SENSU,PRÁTICAS DE IMPROVISAÇÃO PARA A CENA DAS DANÇA...,DEPARTAMENTO DE ARTES,CONSOLIDADA,40.0,7M2345 7T3456
4,57603192,5752917.0,MUT491,TÉCNICO,ARRANJOS II,ESCOLA DE MÚSICA,CONSOLIDADA,20.0,3T34
5,57603193,35466.0,MUT102,TÉCNICO,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,3T456


In [7]:
# filtrando as turmas com código IMD
turmas_imd = merge_turmas_disciplinas[merge_turmas_disciplinas.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd = turmas_imd[turmas_imd.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_imd.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_imd.capacidade_aluno = turmas_imd.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd.id_servidor = turmas_imd.id_servidor.astype(np.int64)

In [72]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

colunas = ["id_turma", "nome_y", "codigo", "nivel_ensino",
          "nome_x","capacidade_aluno", "descricao_horario"]
turmas_imd_nomesprof = pd.merge(turmas_imd, docentes, on='id_servidor')[colunas]
turmas_imd_nomesprof.head()

,id_turma,nome_y,codigo,nivel_ensino,nome_x,capacidade_aluno,descricao_horario
0,57608376,LUCELIO DANTAS DE AQUINO,IMD0017,GRADUAÇÃO,PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS I,60,5N34 (24/07/2017 - 16/12/2017)
1,57608398,LUCELIO DANTAS DE AQUINO,IMD0027,GRADUAÇÃO,PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS II,60,3N34 (24/07/2017 - 16/12/2017)
2,57608691,LUCELIO DANTAS DE AQUINO,IMD0335,GRADUAÇÃO,TÓPICOS ESPECIAIS I,40,35T56 (24/07/2017 - 16/12/2017)
3,57608377,LOURENA KARIN DE MEDEIROS ROCHA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,65,246M3456 (24/07/2017 - 16/12/2017)
4,57608377,CHARLES ANDRYE GALVAO MADEIRA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,65,246M3456 (24/07/2017 - 16/12/2017)


In [73]:
# filtrar apenas as turmas do IMD
notas_imd = notas[notas.id_turma.isin(turmas_imd_nomesprof.id_turma)]

# merge entre os dataframes notas_imd e turmas_imd_nomesprof utilizando como referencia id_turma
notas_imd_merge = pd.merge(notas_imd, turmas_imd_nomesprof, on='id_turma')
notas_imd_merge

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao,nome_y,codigo,nivel_ensino,nome_x,capacidade_aluno,descricao_horario
0,57608383,8488e52d4ca947f5504905cb261636ca,92127264.0,3.0,0.0,f,0.0,1.3,0.0,CANCELADO,ISAAC FRANCO FERNANDES,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,40,24T34 (24/07/2017 - 16/12/2017)
1,57608383,8488e52d4ca947f5504905cb261636ca,92127264.0,3.0,0.0,f,0.0,1.3,0.0,CANCELADO,JULIO CESAR PAULINO DE MELO,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,40,24T34 (24/07/2017 - 16/12/2017)
2,57608383,8488e52d4ca947f5504905cb261636ca,92127264.0,2.0,0.0,f,0.0,1.3,0.0,CANCELADO,ISAAC FRANCO FERNANDES,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,40,24T34 (24/07/2017 - 16/12/2017)
3,57608383,8488e52d4ca947f5504905cb261636ca,92127264.0,2.0,0.0,f,0.0,1.3,0.0,CANCELADO,JULIO CESAR PAULINO DE MELO,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,40,24T34 (24/07/2017 - 16/12/2017)
4,57608383,8488e52d4ca947f5504905cb261636ca,92127264.0,1.0,3.9,f,0.0,1.3,0.0,CANCELADO,ISAAC FRANCO FERNANDES,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,40,24T34 (24/07/2017 - 16/12/2017)
5,57608383,8488e52d4ca947f5504905cb261636ca,92127264.0,1.0,3.9,f,0.0,1.3,0.0,CANCELADO,JULIO CESAR PAULINO DE MELO,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,40,24T34 (24/07/2017 - 16/12/2017)
6,57608383,6979f555ed7b878d55f9cd3fa4e5c032,92127264.0,2.0,0.0,f,0.0,1.8,0.0,CANCELADO,ISAAC FRANCO FERNANDES,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,40,24T34 (24/07/2017 - 16/12/2017)
7,57608383,6979f555ed7b878d55f9cd3fa4e5c032,92127264.0,2.0,0.0,f,0.0,1.8,0.0,CANCELADO,JULIO CESAR PAULINO DE MELO,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,40,24T34 (24/07/2017 - 16/12/2017)
8,57608383,6979f555ed7b878d55f9cd3fa4e5c032,92127264.0,1.0,5.3,f,0.0,1.8,0.0,CANCELADO,ISAAC FRANCO FERNANDES,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,40,24T34 (24/07/2017 - 16/12/2017)
9,57608383,6979f555ed7b878d55f9cd3fa4e5c032,92127264.0,1.0,5.3,f,0.0,1.8,0.0,CANCELADO,JULIO CESAR PAULINO DE MELO,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,40,24T34 (24/07/2017 - 16/12/2017)


In [60]:
def aprovado(column):
    if column["descricao"]==("APROVADO" or "APROVADO POR NOTA"):
        return 1
    else:
        return 0

In [103]:
# Colocando uma flag nos alunos aprovados em 2017.2
rotulos = notas_imd_merge.apply(aprovado, axis = 1)
notas_imd_merge["aprovado"] = rotulos


In [106]:
#Separando apenas os alunos de graduação em 2017.2
graduacao_imd = notas_imd_merge[notas_imd_merge["nivel_ensino"] != "LATO SENSU"]
aprovados_imd = graduacao_imd.pivot_table(index = ["nome_x","nome_y"], values = "aprovado", aggfunc = np.sum)